In [1]:
import pandas as pd
df_graph = pd.read_csv('df_graph_with_wikidata.tsv',sep='\t')
df_graph.head()

,person1,person2,document_ids,wikidata_id1,wikidata_description1,wikidata_id2,wikidata_description2,weight,countries1,countries2,main_country1,main_country2
0,Sybren Valkema,Wim Crouwel,['0001246763-0001246764-0001246765-0001246766-...,Q2618110,Dutch glass artist (1916-1996),Q555389,"Dutch graphic designer, type designer, and typ...",4,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands
1,Bert Haanstra,Sybren Valkema,['0001246763-0001246764-0001246765-0001246766-...,Q827371,Dutch film director (1916-1997),Q2618110,Dutch glass artist (1916-1996),10,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands
2,Jane Bruce,Sybren Valkema,['0001246763-0001246764-0001246765-0001246766-...,Q16031419,Canadian teacher,Q2618110,Dutch glass artist (1916-1996),18,[],['Kingdom of the Netherlands'],NaN,Kingdom of the Netherlands
3,Frits Gramberg,Sybren Valkema,['0001246763-0001246764-0001246765-0001246766-...,Q42429178,Description not found.,Q2618110,Dutch glass artist (1916-1996),19,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands
4,Bert Haanstra,Wim Crouwel,['0001246763-0001246764-0001246765-0001246766-...,Q827371,Dutch film director (1916-1997),Q555389,"Dutch graphic designer, type designer, and typ...",1,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands


In [15]:
import networkx as nx
df_graph_common = df_graph[df_graph['weight']>1]

G = nx.Graph()
for _,row in df_graph.iterrows():
    G.add_edge(row['person1'],row['person2'],weight=row['weight'])

In [62]:
import networkx as nx
from sklearn.preprocessing import MinMaxScaler

def top_n_centrality(G,N=10):
    # Calculate centralities
    degree_centrality = nx.degree_centrality(G)
    weighted_degree = dict(G.degree(weight='weight')) 
    betweenness_centrality = nx.betweenness_centrality(G)# not adding weight here because it's interpreted as distance
    closeness_centrality = nx.closeness_centrality(G)
    largest_component_subgraph = G.subgraph(max(nx.connected_components(G),key=len)).copy()
    pagerank = pnx.pagerank(largest_component_subgraph)

    
    # Adjust max_iter and tol to help with convergence
    try:
        eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=1000, tol=1e-4)
    except nx.PowerIterationFailedConvergence:
        print("Eigenvector centrality did not converge.")


    # Create a DataFrame for centrality metrics
    centrality_df = pd.DataFrame({
        'degree': degree_centrality,
        'weighted_degree':weighted_degree,
        'betweenness': betweenness_centrality,
        'closeness': closeness_centrality,
        'eigenvector': eigenvector_centrality,
        'pagerank':pagerank,
    })

    # Normalize centralities using MinMaxScaler
    scaler = MinMaxScaler()
    normalized_df = pd.DataFrame(scaler.fit_transform(centrality_df), columns=centrality_df.columns, index=centrality_df.index)

    # Compute a composite score by summing the normalized centralities
    normalized_df['composite'] = normalized_df.sum(axis=1)

    # Sort nodes by the composite score
    sorted_nodes = normalized_df.sort_values(by='composite', ascending=False).round(3)


    return sorted_nodes[:N]

In [63]:
top_n_centrality(G,10)

NameError: name 'pnx' is not defined

In [ ]:
# Now, the manual graph for comparison

In [18]:
df_graph_manual = pd.read_csv('/ivi/ilps/personal/vprovat/valkema_graph_directed_manual_FINAL.tsv',sep='\t')

In [19]:
df_graph_manual['weight']=df_graph_manual['document_ids'].apply(lambda x: len(eval(x)))
df_graph_manual

,person1,person2,document_ids,weight
0,J. (Han) Feenstra,Sybren Valkema,['1310445'],1
1,Jan Antonie de Melker,Sybren Valkema,"['1262435', '1262439', '1262454']",3
2,A. Binter,Sybren Valkema,"['1269270', '1269271', '1269272']",3
3,A. Binter,Veronique Valkema,"['1269270', '1269271', '1269272']",3
4,A. Jansen,Sybren Valkema,"['1459844', '1459845', '1459846', '1459847']",4
...,...,...,...,...
254,Sybren Valkema,sochar Rajko Georgiev Rajkov,"['1453564', '1453571', '1453572', '1453573', '...",9
255,Sybren Valkema,students,"['1269476', '1269477', '1269478']",3
256,Sybren Valkema,wy herman hart,['1297980'],1
257,Veronique Valkema,Zaharah Hughes,['1288898'],1


In [68]:
# def split_name(name): # for cases like Sybren and Veronique Valkema
#     if name.find(' and ') == -1:
#         return [name]
    
#     first_part, second_part = name.split(' and ') # only two people together, not more
    
#     if second_part.find(' ') != -1: # last name present
#         if first_part.find(' ') == -1: # no last name for first person
#             lastname = second_part.split(' ')[-1]
#             first_part += ' '+lastname
#     return [first_part,second_part]

In [69]:
# split_name('Sybren and Veronique Valkema')

['Sybren Valkema', 'Veronique Valkema']

In [20]:
G_manual = nx.Graph()

for _,row in df_graph_manual.iterrows():
    person1, person2 = row['person1'],row['person2']
    
#     if person1.startswith('?') or person2.startswith('?'):
#         continue
#     if person2 == 'Sybren  Valkema':
#         person2 = 'Sybren Valkema'
        
    weight=row['weight']
    if weight > 1:
#     people1, people2 = split_name(person1),split_name(person2)
#     for per1 in people1:
#         for per2 in people2:
        G_manual.add_edge(person1, person2,weight=weight)

In [21]:
top_n_centrality(G_manual,10)

,degree,weighted_degree,betweenness,closeness,eigenvector,composite
Sybren Valkema,1.000,1.000,1.000,1.000,1.000,5.000
Veronique Valkema,0.161,0.173,0.012,0.548,0.377,1.271
Durk Valkema,0.025,0.074,0.000,0.514,0.381,0.996
Jaroslava Brychtova,0.017,0.046,0.008,0.512,0.282,0.865
Stanislav Libensky,0.017,0.044,0.008,0.512,0.270,0.852
Charles Bray,0.008,0.032,0.000,0.508,0.247,0.796
Ingeborg Lundin,0.008,0.032,0.000,0.508,0.211,0.759
Albert Lewis,0.000,0.026,0.000,0.506,0.222,0.754
Kl.Laansma,0.008,0.026,0.017,0.510,0.188,0.748
Arjan Muijlwijk,0.008,0.032,0.000,0.508,0.196,0.745


In [ ]:
# Now, the graph created automatically on the same sample as the manual grapjh

In [22]:
df_graph_small = pd.read_csv('df_graph_same_docs_as_manual.tsv',sep='\t')

In [24]:
G_small = nx.Graph()

for _,row in df_graph_small.iterrows():
    person1, person2 = row['person1'],row['person2']
    
        
    weight=row['weight']
    if weight > 1:
#     people1, people2 = split_name(person1),split_name(person2)
#     for per1 in people1:
#         for per2 in people2:
        G_small.add_edge(person1, person2,weight=weight)

In [25]:
top_n_centrality(G_small,10)

,degree,weighted_degree,betweenness,closeness,eigenvector,composite
Sybren Valkema,1.000,1.000,1.000,1.000,1.000,5.000
Harvey Littleton,0.393,0.360,0.126,0.550,0.626,2.055
Albert Lewis,0.321,0.324,0.031,0.431,0.503,1.611
Durk Valkema,0.286,0.180,0.255,0.458,0.154,1.334
Gerrit Rietveld,0.143,0.180,0.121,0.367,0.487,1.299
Erwin Eisch,0.214,0.225,0.013,0.392,0.422,1.267
Sam Herman,0.286,0.189,0.085,0.417,0.283,1.260
Finn Lynggaard,0.143,0.090,0.007,0.344,0.193,0.776
Claude Morin,0.143,0.081,0.002,0.367,0.177,0.770
Mieke Groot,0.143,0.090,0.038,0.392,0.089,0.752


In [40]:
%
import sys
!{sys.executable} -m pip uninstall community -y
!{sys.executable} -m pip uninstall python-louvain -y
!{sys.executable} -m pip install python-louvain

Uninstalling python-louvain-0.16:
  Successfully uninstalled python-louvain-0.16
Processing /home/vprovat/.cache/pip/wheels/73/68/16/01a6fa0a2f8a718cda81c3b8108d5afbc9feb1030d053e5532/python_louvain-0.16-cp37-none-any.whl


In [60]:
# Now, graph-wide statistics for everyone

import community.community_louvain as community_louvain

def get_graph_metrics(G_raw,graph_name):
    partition = community_louvain.best_partition(G_raw)
    G = G_raw.subgraph(max(nx.connected_components(G_raw),key=len)).copy() # if the graph is not connected, work with largest component
    return {
        'graph_name':graph_name,
        'num_components': nx.number_connected_components(G_raw),
        'num_nodes': G.number_of_nodes(),
        'avg_degree': sum(dict(G.degree()).values()) / G.number_of_nodes(),
        'avg_weighted_degree': sum(dict(G.degree(weight='weight')).values()) / G.number_of_nodes(),
        'density': nx.density(G),
        'diameter': nx.diameter(G),
        'avg_clustering': nx.average_clustering(G),
#         'partition': partition,
        'modularity': community_louvain.modularity(partition, G)
    }
        
records=[get_graph_metrics(G,'graph'),
         get_graph_metrics(G_manual,'graph_manual'),
         get_graph_metrics(G_small,'graph_small')]

In [61]:
df_graph_stats = pd.DataFrame.from_records(records)
df_graph_stats.round(2)

,graph_name,num_components,num_nodes,avg_degree,avg_weighted_degree,density,diameter,avg_clustering,modularity
0,graph,1,868,86.54,306.20,0.10,5,0.85,0.42
1,graph_manual,3,124,2.35,12.74,0.02,4,0.17,0.17
2,graph_small,1,43,3.86,11.67,0.09,4,0.49,0.36


In [57]:
pagerank = nx.pagerank(G)
print(f"PageRank: {pagerank}")


PageRank: {'Sybren Valkema': 0.0329677638539127, 'Wim Crouwel': 0.00027315721551213754, 'Bert Haanstra': 0.0012841298440811994, 'Jane Bruce': 0.001606003301308217, 'Frits Gramberg': 0.0009260173887085368, 'William Fine': 0.00018529792000306398, 'van der Linden': 0.005019203196325896, 'Durk Valkema': 0.010787479308607932, 'Gerrit Rietveld': 0.011957230279594765, 'Walter Lieberman': 0.004699442969675591, 'Mike Rowland': 0.00021513699224473342, 'Jan Oosterman': 0.00034914990348564295, 'Charles Johnson': 0.0015950323848632193, 'Erwin Eisch': 0.008719835412393406, 'Robert Fritz': 0.001843843418158153, 'Margaret Pennington': 0.0012370280449767127, 'Sam Herman': 0.0038882271308637324, 'Richard Meitner': 0.007859783355986309, 'John Cook': 0.0019981083183107386, 'Bert van Loo': 0.00305326724662021, 'Pauline Wiertz': 0.0006858019620213627, 'Hasse Holmerud': 0.006265314217155115, 'de Jong': 0.0019450376940145827, 'Victor Onstein': 0.0006226442052133295, 'Mieke Groot': 0.006044516300482157, 'Gerar

In [ ]:
# Old code begins here

In [21]:
def metrics_to_df(metrics,metric_name='degree_centrality'):
    return pd.DataFrame.from_records(sorted(metrics.items(), key=lambda item: item[1], reverse=True),
                                                    columns=['name',metric_name
                                                            ]).round(3)

In [24]:
degree_centrality_df = metrics_to_df(nx.degree_centrality(G))
betweenness_centrality_df = metrics_to_df(nx.betweenness_centrality(G),'betweenness_centrality')
closeness_centrality_df = metrics_to_df(nx.closeness_centrality(G),'closeness_centrality')
eigenvector_centrality_df = metrics_to_df(nx.eigenvector_centrality(G),'eigenvector_centrality')

eigenvector_centrality_df

,name,eigenvector_centrality
0,Sybren Valkema,0.296
1,Harvey Littleton,0.225
2,Dale Chihuly,0.188
3,Richard Jolley,0.166
4,Lino Tagliapietra,0.161
...,...,...
393,Carolyn Adams,0.000
394,Monica Guggisberg,0.000
395,Philip Baldwin,0.000
396,Adriek Westenenk,0.000


In [10]:
sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)

[('Sybren Valkema', 0.5012594458438288),
 ('Richard Jolley', 0.43324937027707805),
 ('Harvey Littleton', 0.2947103274559194),
 ('Dale Chihuly', 0.2947103274559194),
 ('John Anderson', 0.22921914357682618),
 ('Kea Verwey', 0.21158690176322417),
 ('Gerrit Rietveld', 0.20906801007556675),
 ('Lino Tagliapietra', 0.20151133501259444),
 ('Claude Morin', 0.19395465994962216),
 ('Durk Valkema', 0.18639798488664988),
 ('Alice Rooney', 0.17632241813602015),
 ('Willem Heesen', 0.17380352644836272),
 ('Ulla Forsell', 0.16876574307304784),
 ('Erwin Eisch', 0.16624685138539042),
 ('Hasse Holmerud', 0.16624685138539042),
 ('Charles Parriott', 0.15365239294710328),
 ('Richard Meitner', 0.14357682619647355),
 ('Joel Myers', 0.14357682619647355),
 ('Walter Lieberman', 0.14105793450881612),
 ('Helmut Ricke', 0.12090680100755667),
 ('John Hauberg', 0.12090680100755667),
 ('Joop van den Broek', 0.11083123425692695),
 ('Bertil Vallien', 0.11083123425692695),
 ('Mieke Groot', 0.10831234256926951),
 ('Michael